In [1]:
from multiprocessing import Pool

def f(x):
    return x*x

In [2]:
with Pool(5) as p:
    print(p.map(f, [1, 2, 3]))

[1, 4, 9]


In [3]:
from multiprocessing import Process

def f(name):
    print('hello', name)

p = Process(target=f, args=('bob',))
p.start()
p.join()

hello bob


In [4]:
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)


info('main line')
p = Process(target=f, args=('bob',))
p.start()
p.join()

main line
module name: __main__
parent process: 11316
process id: 50564
function f
module name: __main__
parent process: 50564
process id: 50612
hello bob


In [ ]:
# restart the notebook before executing next cell, otherwise you'll get an error 'context already set'

In [1]:
import multiprocessing as mp

def foo(q):
    q.put('hello')

#if __name__ == '__main__':
#mp.set_start_method('spawn')
mp.set_start_method('fork')
q = mp.Queue()
p = mp.Process(target=foo, args=(q,))
p.start()
print(q.get())
p.join()

# this is actually never ending !? works with 'fork', but not with 'spawn'
# problem of the notebook

hello


In [2]:
ctx = mp.get_context('fork')
q = ctx.Queue()
p = ctx.Process(target=foo, args=(q,))
p.start()
print(q.get())
p.join()

# this is actually never ending !? works with 'fork', but not with 'spawn'

hello


In [3]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

##if __name__ == '__main__':
q = Queue()
p = Process(target=f, args=(q,))
p.start()
print(q.get())    # prints "[42, None, 'hello']"
p.join()

[42, None, 'hello']


In [4]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    conn.close()

#if __name__ == '__main__':
parent_conn, child_conn = Pipe()
p = Process(target=f, args=(child_conn,))
p.start()
print(parent_conn.recv())   # prints "[42, None, 'hello']"
p.join()

[42, None, 'hello']


In [8]:
from multiprocessing import Process, Lock

def f(l, i):
    l.acquire()
    try:
        print('hello world', i)
    finally:
        l.release()

#if __name__ == '__main__':
lock = Lock()

for num in range(10):
    Process(target=f, args=(lock, num)).start()
    
# they're already/still mixed... probably works in a terminal -> yes

hello world 1
hello world 0
hello world 3
hello world 2
hello world 4
hello world 5
hello world 6
hello world 8
hello world 7
hello world 9


Server process managers are more flexible than using shared memory objects because they can be made to support arbitrary object types. Also, a single manager can be shared by processes on different computers over a network. They are, however, slower than using shared memory.

Bear in mind that a process that has put items in a queue will wait before terminating until all the buffered items are fed by the “feeder” thread to the underlying pipe. 

This means that whenever you use a queue you need to make sure that all items which have been put on the queue will eventually be removed before the process is joined. Otherwise you cannot be sure that processes which have put items on the queue will terminate. Remember also that non-daemonic processes will be joined automatically.

On Unix using the fork start method, a child process can make use of a shared resource created in a parent process using a global resource. However, it is better to pass the object as an argument to the constructor for the child process.